# Eksperimen Klasifikasi Phishing: XGBoost

**Tujuan Notebook:**
1. Memuat dan melakukan pra-pemrosesan pada dataset deteksi phishing.
2. Melakukan hyperparameter tuning pada model XGBoost menggunakan `RandomizedSearchCV`.
3. Melatih model XGBoost final dengan parameter terbaik.
4. Mengevaluasi performa model pada test set.
5. Menyimpan metrik hasil evaluasi dan model terlatih untuk analisis lebih lanjut.

## Setup Environment

In [1]:
# Mount Google Drive (jika belum)
try:
    from google.colab import drive
    drive.mount('/content/drive')
except ImportError:
    print("Bukan di lingkungan Colab, pastikan path sudah benar.")

Mounted at /content/drive


## Import Library dan Konfigurasi

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time
import json
import os

print(f"Versi XGBoost: {xgb.__version__}")

Versi XGBoost: 2.1.4


In [3]:
# Sesuaikan path ini dengan lokasi file Anda di Google Drive
# Pastikan Google Drive sudah di-mount
BASE_PATH = "/content/drive/My Drive/datasets/Phishing_Detection_Dataset/"
TRAINING_DATA_PATH = os.path.join(BASE_PATH, "dataset_training.csv")
TESTING_DATA_PATH = os.path.join(BASE_PATH, "dataset_testing.csv")
OUTPUT_DIR = os.path.join(BASE_PATH, "outputs")

# Membuat direktori output jika belum ada
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Konfigurasi untuk reproduktivitas
RANDOM_STATE = 6037

## Preprocessing Data

In [4]:
# --- Memuat Data Training dan Testing ---
df_train_full = pd.read_csv(TRAINING_DATA_PATH)
df_test = pd.read_csv(TESTING_DATA_PATH)

print(f"Jumlah data training (keseluruhan): {len(df_train_full)}")
print(f"Jumlah data testing (dari file terpisah): {len(df_test)}")

# --- Memisahkan Fitur dan Target ---
# Data dari file training
X_train_full = df_train_full.drop(columns=["Type"]).astype(np.float32)
y_train_full = df_train_full["Type"].astype(int)

# Data dari file testing
X_test = df_test.drop(columns=["Type"]).astype(np.float32)
y_test = df_test["Type"].astype(int)

# --- Membuat Set Validasi dari Data Training ---
# Kita membagi data training menjadi set training (untuk melatih) dan validasi (untuk tuning)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full,
    test_size=0.2, # 20% dari data training akan digunakan untuk validasi
    stratify=y_train_full,
    random_state=RANDOM_STATE
)

print(f"\nUkuran data training (untuk tuning): {X_train.shape}")
print(f"Ukuran data validasi (untuk tuning): {X_val.shape}")
print(f"Ukuran data test (final):            {X_test.shape}")

# --- Penskalaan Fitur ---
# Scaler HANYA "belajar" (di-fit) dari data training (X_train)
scaler_xgb = MinMaxScaler()
scaler_xgb.fit(X_train)

# Terapkan scaler yang sama ke semua set data
X_train_xgb = scaler_xgb.transform(X_train)
X_val_xgb = scaler_xgb.transform(X_val)
X_test_xgb = scaler_xgb.transform(X_test) # Data test juga di-transform dengan scaler dari data train

# Kita juga perlu men-transform X_train_full untuk pelatihan model final nanti
X_train_full_xgb = scaler_xgb.transform(X_train_full)

Jumlah data training (keseluruhan): 223155
Jumlah data testing (dari file terpisah): 24795

Ukuran data training (untuk tuning): (178524, 41)
Ukuran data validasi (untuk tuning): (44631, 41)
Ukuran data test (final):            (24795, 41)


## Hyperparameter Tuning

In [5]:
print("Memulai Hyperparameter Tuning untuk XGBoost...")

# Inisialisasi model dasar dengan dukungan GPU
xgb_base = xgb.XGBClassifier(
    tree_method='hist', device='cuda', eval_metric='logloss', random_state=RANDOM_STATE
)

# Definisikan ruang pencarian parameter
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [3, 5, 7, 9, 11],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'gamma': [0, 0.1, 0.2, 0.3],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

# Setup RandomizedSearchCV
xgb_search = RandomizedSearchCV(
    estimator=xgb_base,
    param_distributions=param_dist,
    n_iter=30, # Jumlah kombinasi yang akan dicoba
    scoring='f1', # Optimalkan berdasarkan F1-score
    cv=3, # 3-fold cross-validation
    verbose=2,
    random_state=RANDOM_STATE,
    n_jobs=-1 # Gunakan semua core yang tersedia
)

# Lakukan pencarian pada data training
xgb_search.fit(X_train_xgb, y_train)

print("\nTuning selesai.")
print(f"Best F1-score from CV: {xgb_search.best_score_:.4f}")
print(f"Best XGBoost params: {xgb_search.best_params_}")

Memulai Hyperparameter Tuning untuk XGBoost...
Fitting 3 folds for each of 30 candidates, totalling 90 fits

Tuning selesai.
Best F1-score from CV: 0.9406
Best XGBoost params: {'subsample': 0.6, 'n_estimators': 500, 'max_depth': 11, 'learning_rate': 0.1, 'gamma': 0.2, 'colsample_bytree': 0.6}


## Evaluasi Best Model

In [13]:
# 1. Ambil parameter terbaik dari hasil tuning
best_params = xgb_search.best_params_

# 2. Buat instance model baru HANYA untuk membuat histori
# Pastikan device diatur ke 'cuda' untuk konsistensi
history_model = xgb.XGBClassifier(
    **best_params,
    tree_method='hist',
    device='cuda',
    eval_metric=['logloss', 'error'], # Minta logloss dan error
    random_state=RANDOM_STATE
)

# 3. Latih model ini pada data training, dan evaluasi pada data validasi
# Ini adalah langkah kunci untuk mendapatkan kurva train vs. val
history_model.fit(
    X_train_xgb,
    y_train,
    eval_set=[(X_train_xgb, y_train), (X_val_xgb, y_val)],
    verbose=False
)

# 4. Ambil dan simpan hasilnya
xgb_history = history_model.evals_result()
history_path = os.path.join(OUTPUT_DIR, 'xgb_history.json')

with open(history_path, 'w') as f:
    json.dump(xgb_history, f, indent=4)

print(f"Data histori untuk grafik berhasil dibuat dan disimpan di: {history_path}")

Data histori untuk grafik berhasil dibuat dan disimpan di: /content/drive/My Drive/datasets/Phishing_Detection_Dataset/outputs/xgb_history.json


## Training Model Final

In [17]:
print("Melatih model final XGBoost dengan parameter terbaik pada SELURUH data training...")

# Ambil model dengan parameter terbaik dari hasil tuning
best_xgb_model = xgb_search.best_estimator_

# Latih model final pada seluruh data training yang tersedia (dari file dataset_training.csv)
# Data ini sudah di-scale dengan benar pada langkah sebelumnya.
best_xgb_model.fit(X_train_full_xgb, y_train_full)
# best_xgb_model.save_model(os.path.join(OUTPUT_DIR, 'best_xgb_model.json'))

print("Pelatihan model final pada seluruh data selesai.")

Melatih model final XGBoost dengan parameter terbaik pada SELURUH data training...
Pelatihan model final pada seluruh data selesai.


## Evaluasi Model

In [15]:
model_path = os.path.join(OUTPUT_DIR, 'best_xgb_model.json')

### Inferensi di CPU

In [9]:
# ==============================================================================
#                              Evaluasi di CPU
# ==============================================================================
print("\n--- Memulai Evaluasi di CPU ---")

# Buat instance model baru yang secara eksplisit disetel untuk berjalan di CPU
cpu_xgb_model = xgb.XGBClassifier(device='cpu')
# Muat bobot dari model terbaik yang sudah kita simpan
cpu_xgb_model.load_model(model_path)

# Lakukan prediksi pada data test di CPU (X_test_xgb adalah numpy array)
y_pred_xgb_cpu = cpu_xgb_model.predict(X_test_xgb)

# Hitung metrik performa
xgb_cpu_metrics = {
    'Model': 'XGBoost (Tuned)',
    'Device': 'CPU',
    'Accuracy': accuracy_score(y_test, y_pred_xgb_cpu),
    'Precision': precision_score(y_test, y_pred_xgb_cpu),
    'Recall': recall_score(y_test, y_pred_xgb_cpu),
    'F1-score': f1_score(y_test, y_pred_xgb_cpu)
}

# Ukur kecepatan inferensi di CPU (rata-rata dari 50 eksekusi)
cpu_inference_times = []
for _ in range(50):
    start_time = time.perf_counter()
    _ = cpu_xgb_model.predict(X_test_xgb)
    end_time = time.perf_counter()
    cpu_inference_times.append((end_time - start_time) * 1000)

# Waktu inferensi rata-rata per sampel dalam milidetik
xgb_cpu_metrics['Inference Time (ms/sample)'] = np.mean(cpu_inference_times) / len(X_test_xgb)

print("Metrik Kinerja di CPU:")
for key, value in xgb_cpu_metrics.items():
    if isinstance(value, float):
        print(f"{key:<25}: {value:.6f}")
    else:
        print(f"{key:<25}: {value}")


--- Memulai Evaluasi di CPU ---
Metrik Kinerja di CPU:
Model                    : XGBoost (Tuned)
Device                   : CPU
Accuracy                 : 0.955031
Precision                : 0.967912
Recall                   : 0.939059
F1-score                 : 0.953267
Inference Time (ms/sample): 0.026744


### Inferensi di GPU

In [10]:
# ==============================================================================
#                Evaluasi di GPU
# ==============================================================================
print("--- Memulai Evaluasi di GPU ---")

try:
    import cupy as cp
    # Pindahkan data test ke GPU menggunakan CuPy
    X_test_gpu = cp.array(X_test_xgb)

    # Lakukan prediksi pada data test di GPU
    y_pred_xgb_gpu = best_xgb_model.predict(X_test_gpu)
    # Pindahkan hasil prediksi kembali ke CPU (numpy) untuk dihitung metriknya
    y_pred_xgb_gpu_np = cp.asnumpy(y_pred_xgb_gpu)

    # Hitung metrik performa
    xgb_gpu_metrics = {
        'Model': 'XGBoost (Tuned)',
        'Device': 'GPU',
        'Accuracy': accuracy_score(y_test, y_pred_xgb_gpu_np),
        'Precision': precision_score(y_test, y_pred_xgb_gpu_np),
        'Recall': recall_score(y_test, y_pred_xgb_gpu_np),
        'F1-score': f1_score(y_test, y_pred_xgb_gpu_np)
    }

    # Ukur kecepatan inferensi di GPU (rata-rata dari 50 eksekusi)
    gpu_inference_times = []
    for _ in range(50):
        start_time = time.perf_counter()
        _ = best_xgb_model.predict(X_test_gpu)
        end_time = time.perf_counter()
        gpu_inference_times.append((end_time - start_time) * 1000)

    # Waktu inferensi rata-rata per sampel dalam milidetik
    xgb_gpu_metrics['Inference Time (ms/sample)'] = np.mean(gpu_inference_times) / len(X_test_xgb)

    print("Metrik Kinerja di GPU:")
    for key, value in xgb_gpu_metrics.items():
        if isinstance(value, float):
          print(f"{key:<25}: {value:.6f}")
        else:
          print(f"{key:<25}: {value}")

except ImportError:
    print("CuPy tidak terinstal. Melewatkan evaluasi GPU.")
    xgb_gpu_metrics = {}




--- Memulai Evaluasi di GPU ---
Metrik Kinerja di GPU:
Model                    : XGBoost (Tuned)
Device                   : GPU
Accuracy                 : 0.955031
Precision                : 0.967912
Recall                   : 0.939059
F1-score                 : 0.953267
Inference Time (ms/sample): 0.000804


## Simpan Model

In [18]:
# Tentukan path file output
metrics_path = os.path.join(OUTPUT_DIR, 'xgb_results.json')
# Path model tidak berubah
# model_path = os.path.join(OUTPUT_DIR, 'best_xgb_model.json')

history_path = os.path.join(OUTPUT_DIR, 'xgb_history.json')

# Gabungkan kedua metrik ke dalam satu dictionary
final_results = {
    'gpu_metrics': xgb_gpu_metrics,
    'cpu_metrics': xgb_cpu_metrics
}

# Simpan metrik gabungan ke file JSON
with open(metrics_path, 'w') as f:
    json.dump(final_results, f, indent=4)

# Simpan model terlatih (model GPU asli sudah cukup, karena bobotnya sama)
best_xgb_model.save_model(model_path) # sudah dilakukan di sel sebelumnya jika Anda memisahkannya

print(f"\nMetrik evaluasi (CPU & GPU) telah disimpan di: {metrics_path}")
print(f"Model terbaik (dilatih di GPU) disimpan di: {model_path}")
# print(f"Histori training telah disimpan di: {history_path}")


Metrik evaluasi (CPU & GPU) telah disimpan di: /content/drive/My Drive/datasets/Phishing_Detection_Dataset/outputs/xgb_results.json
Model terbaik (dilatih di GPU) disimpan di: /content/drive/My Drive/datasets/Phishing_Detection_Dataset/outputs/best_xgb_model.json
